<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#!pip install onnx onnxruntime


In [2]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input_name_A = "A"
input_name_B = "B"
less_output_name = "LessOutput"

#-------------------------------------------------------------------------------
# 2-rank input tensors
#-------------------------------------------------------------------------------

# Create 2-rank input tensors
input_tensor_A = onnx.helper.make_tensor_value_info(
    input_name_A, onnx.TensorProto.FLOAT, [None, None]
)
input_tensor_B = onnx.helper.make_tensor_value_info(
    input_name_B, onnx.TensorProto.FLOAT, [None, None]
)

# Create output tensor (result after Less operation)
less_output = onnx.helper.make_tensor_value_info(
    less_output_name, onnx.TensorProto.BOOL, [None, None]
)

# Define Less node: C = Less(A, B)
less_node = onnx.helper.make_node(
    "Less",
    [input_name_A, input_name_B],
    [less_output_name]
)

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[less_node],
    name="Less",
    inputs=[input_tensor_A, input_tensor_B],
    outputs=[less_output]
)

# Create the ONNX model (Less is available in opset 13+; we use 16 here)
model = onnx.helper.make_model(
    graph,
    opset_imports=[onnx.helper.make_opsetid("", 16)]
)
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "less.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("less.onnx")

def do_less(a, b):
    # Run inference
    output = session.run(None, {input_name_A: a, input_name_B: b})

    a_f = (np.array2string(a, separator=',', max_line_width=np.inf).replace('\n', ''))
    b_f = (np.array2string(b, separator=',', max_line_width=np.inf).replace('\n', ''))
    o_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', ''))

    # Display results
    print(f"A={a_f}")
    print(f"B={b_f}")
    print(f"Result: Less(A,B)={o_f}")

np.set_printoptions(precision=None, floatmode='fixed')

#--------------------------------------------------------------------
# Nominal cases
#--------------------------------------------------------------------

# Case N1: 2-rank tensors (float32)
a = np.array([[2.0, 3.0, 7.0]], dtype=np.float32)
b = np.array([[3.0, 3.0, 5.0]], dtype=np.float32)
do_less(a, b)

a = np.array([[1.0, 2.0],
              [4.0, 0.0],
              [5.0, 6.0]], dtype=np.float32)

b = np.array([[3.0, 2.0],
              [4.0, 1.0],
              [5.0, 4.0]], dtype=np.float32)

do_less(a, b)

#--------------------------------------------------------------------
# Non nominal cases (nan, +inf, -inf)
#--------------------------------------------------------------------

vA = [1.0, float("nan"), float("inf"), float("-inf")]
vB = [1.0, 0.0,        float("inf"),  float("-inf")]

for a_val, b_val in zip(vA, vB):
    a_np = np.array([[a_val],
                     [a_val]], dtype=np.float32)
    b_np = np.array([[b_val],
                     [b_val]], dtype=np.float32)
    do_less(a_np, b_np)

#--------------------------------------------------------------------
# Examples from documentation
#--------------------------------------------------------------------

print("\n## Example 1")

# Example 1
# A = [2.5, 3.7, 7.9]
# B = [3.1, 3.7, 5.8]
a_example_1 = np.array([[2.5, 3.7, 7.9]], dtype=np.float32)
b_example_1 = np.array([[3.1, 3.0, 5.8]], dtype=np.float32)
do_less(a_example_1, b_example_1)

print("\n## Example 2")

# Example 2
# A =
# [[  1.1   2.0 ]
#  [  4.2   0.0 ]
#  [  5.3   6.4 ]]
# B =
# [[  3.5   2 ]
#  [  4.6   1.0 ]
#  [  5.3   4.8 ]]
a_example_2 = np.array([
    [1.1, 2.0],
    [4.2, 0.0],
    [5.3, 6.4]
], dtype=np.float32)

b_example_2 = np.array([
    [3.5, 2.0],
    [4.6, 1.0],
    [5.7, 4.8]
], dtype=np.float32)

do_less(a_example_2, b_example_2)

print("\n## Example 3")

# Example 3
# A = [[  -inf  -inf -inf  -inf   0.0   0.0  0.0  0.0    inf   inf  inf  inf    nan   nan  nan  nan ]]
# B = [[  -inf   0.0  inf  nan   -inf   0.0  inf  nan   -inf   0.0  inf  nan   -inf   0.0  inf  nan ]]
a_example_3 = np.array([
    [float("-inf"), float("-inf"), float("-inf"), float("-inf"), 0.0, 0.0, 0.0, 0.0, float("inf"), float("inf"), float("inf"), float("inf"), float("nan"), float("nan"), float("nan"), float("nan"), ]
], dtype=np.float32)

b_example_3 = np.array([
    [float("-inf"), 0.0, float("inf"), float("nan"), float("-inf"), 0.0, float("inf"), float("nan"), float("-inf"), 0.0, float("inf"), float("nan"), float("-inf"), 0.0, float("inf"), float("nan")         ]
], dtype=np.float32)

do_less(a_example_3, b_example_3)

A=[[2.00000000,3.00000000,7.00000000]]
B=[[3.00000000,3.00000000,5.00000000]]
Result: Less(A,B)=[[ True,False,False]]
A=[[1.00000000,2.00000000], [4.00000000,0.00000000], [5.00000000,6.00000000]]
B=[[3.00000000,2.00000000], [4.00000000,1.00000000], [5.00000000,4.00000000]]
Result: Less(A,B)=[[ True,False], [False, True], [False,False]]
A=[[1.00000000], [1.00000000]]
B=[[1.00000000], [1.00000000]]
Result: Less(A,B)=[[False], [False]]
A=[[nan], [nan]]
B=[[0.00000000], [0.00000000]]
Result: Less(A,B)=[[False], [False]]
A=[[inf], [inf]]
B=[[inf], [inf]]
Result: Less(A,B)=[[False], [False]]
A=[[-inf], [-inf]]
B=[[-inf], [-inf]]
Result: Less(A,B)=[[False], [False]]

## Example 1
A=[[2.50000000,3.70000005,7.90000010]]
B=[[3.09999990,3.00000000,5.80000019]]
Result: Less(A,B)=[[ True,False,False]]

## Example 2
A=[[1.10000002,2.00000000], [4.19999981,0.00000000], [5.30000019,6.40000010]]
B=[[3.50000000,2.00000000], [4.59999990,1.00000000], [5.69999981,4.80000019]]
Result: Less(A,B)=[[ True,Fals